In [1]:
import pandas as pd
import numpy as np
import os
import subprocess

In [8]:
import argparse

In [2]:
def create_and_submit_sbatch(batch, chunk):
    name="RHO_Csec_gene_batch{batch}.job".format(batch=batch)
    sbatch = [ "#!/bin/bash -l",
                    "#SBATCH -A naiss2023-22-450",
                    "#SBATCH -t 2:0:0 ",
                    "#SBATCH -p core -n 1",
                    "#SBATCH -J {name}".format(name=name),
                    "#SBATCH -o {name}_%j.out".format(name=name),
                    "#SBATCH -e {name}_%j.error".format(name=name),
                    "#SBATCH --get-user-env",
                    "##command underneath this##",
                    "conda activate base",
                    "python csec_get_intron_mean.py --genelist {chunk} --batchname {batch}".format(chunk=chunk, batch=batch)]
    with open(name, 'wt') as handle:
        handle.write("\n".join(sbatch))
    subprocess.call(["chmod", "766", name])
    subprocess.call(["sbatch", name])

In [3]:
Csec_rho_gene_and_flank = pd.read_csv('./Csec/20230919_Csec_rho_per_gene_and_50kbflank_with_10kbbuffer_newfilt.tsv', sep='\t')
Csec_rho_exon_and_intron = pd.read_csv('/proj/snic2021-23-365/private/TR_20230707/TR_20230707_per_gene_analysis/Csec/old_data/rho_per_gene/20230717_Csec_rho_exons_and_introns.tsv', sep='\t', index_col=0)
Csec_rho_exon_and_intron.columns = ['scaffold', 'source', 'featuretype', 'start', 'stop', 'n', 'strand', 'n2', 'ID', 'scaffold_alt', 'rho']
        #CpG
Csec_CpG_gene_and_flank = pd.read_csv( './Csec/old_data/20230810_Csec_CpG_per_gene_and_50kbflank_10kbbuffer.tsv', sep='\t', index_col=0)
Csec_CpG_exon_and_intron = pd.read_csv('/proj/snic2021-23-365/private/TR_20230707/TR_20230707_per_gene_analysis/Csec/old_data/20230717_Csec_CpG_exons_and_introns.tsv', sep='\t', index_col=0)

    #Mbel
        #rho



Csec_rho_gene_and_flank = Csec_rho_gene_and_flank.dropna(subset=['weighted_mean_rho_gene', 'weighted_mean_rho_uflank', 'weighted_mean_rho_dflank'], how='any')

Csec_gene_and_flank = pd.merge(left=Csec_rho_gene_and_flank, right=Csec_CpG_gene_and_flank, left_on='idstring', right_on='ID', how='left')

Csec_exin = pd.merge(left=Csec_CpG_exon_and_intron, right=Csec_rho_exon_and_intron, left_on=['ID','featuretype'], right_on=['ID','featuretype'])

# generating the intron positions with Genometools has left the IDstring somewhat sparse, only denoting parent RNA.
# here i use the richer ID-string of the exons to link parent-RNA to the genebank ID

ID_dict = {}
for i in Csec_exin.loc[Csec_exin.featuretype=='exon'].ID:
    idfields = i.split(';')
    idict = {i.split('=')[0]:i.split('=')[1] for i in idfields}
    ID_dict[idict['Parent']] = idict['Dbxref']


def extract_id(idstring, idtype = 'Dbxref', ID_dict=False):
    if ID_dict == False:
        idfields = idstring.split(';')
        idict = {i.split('=')[0]:i.split('=')[1] for i in idfields}
        return idict[idtype].split(',')[0]
    else:
        idfields = idstring.split(';')
        idict = {i.split('=')[0]:i.split('=')[1] for i in idfields}
        try:
            return idict[idtype].split(',')[0]
        except KeyError:
            return ID_dict[idict['Parent']].split(',')[0]


Csec_gene_and_flank['GeneID'] = [extract_id(i) for i in Csec_gene_and_flank.ID]

Csec_exin['GeneID'] = [extract_id(i, ID_dict=ID_dict) for i in Csec_exin.ID]

# remnove superfluous columns
Csec_gene_and_flank = Csec_gene_and_flank[['scaffold', 'weighted_mean_rho_gene', 'weighted_mean_rho_uflank',
       'weighted_mean_rho_dflank', 'ID', 'gene_cpg_e',
       'gene_cpg_o', 'gene_cpg_oe', 'flank_u_cpg_e', 'flank_u_cpg_o',
       'flank_u_cpg_oe', 'flank_d_cpg_e', 'flank_d_cpg_o', 'flank_d_cpg_oe',
       'GeneID']]


# rename scaffold and rho columns to avoid issues down the line
Csec_exin.columns = ['featuretype', 'ID', 'gene_cpg_e', 'gene_cpg_o', 'gene_cpg_oe',
       'scaffold2', 'source', 'start', 'stop', 'n', 'strand', 'n2',
       'scaffold', 'weighted_mean_rho_gene', 'GeneID']

# remnove superfluous columns
Csec_exin = Csec_exin[['featuretype', 'ID', 'gene_cpg_e', 'gene_cpg_o', 'gene_cpg_oe','source', 'start', 'stop', 'n', 'strand', 'n2','scaffold', 'weighted_mean_rho_gene', 'GeneID']]


# propagate filtering to exin_data
Csec_genefilt = dict.fromkeys(Csec_gene_and_flank.GeneID)


In [4]:
Csec_gene_and_flank.GeneID

0        GeneID:111865144
1        GeneID:111865148
2        GeneID:111865145
3        GeneID:111865150
4        GeneID:117282368
               ...       
12750    GeneID:111861604
12751    GeneID:111861606
12752    GeneID:111861608
12753    GeneID:111861600
12754    GeneID:111861607
Name: GeneID, Length: 12755, dtype: object

In [5]:
#split into chunks and write them
chunk_n = 500
ss_split = np.array_split(list(Csec_gene_and_flank.GeneID), chunk_n)

In [10]:
cfiles = []
for i, k in enumerate(ss_split):
    #print(i)
    chunkname = "/home/tilman/termites/TR_20230707/TR_20230707_per_gene_analysis/csec_intron_batch_gene/ss_chunk{chunk_i}.chunk".format(chunk_i=i)
    #with open(chunkname, 'wt') as handle:
        #handle.write('\n'.join(k))
    create_and_submit_sbatch(batch=i, chunk=chunkname)

Submitted batch job 41233142
Submitted batch job 41233143
Submitted batch job 41233144
Submitted batch job 41233145
Submitted batch job 41233146
Submitted batch job 41233147
Submitted batch job 41233148
Submitted batch job 41233149
Submitted batch job 41233150
Submitted batch job 41233151
Submitted batch job 41233152
Submitted batch job 41233153
Submitted batch job 41233154
Submitted batch job 41233155
Submitted batch job 41233156
Submitted batch job 41233157
Submitted batch job 41233158
Submitted batch job 41233159
Submitted batch job 41233160
Submitted batch job 41233161
Submitted batch job 41233162
Submitted batch job 41233163
Submitted batch job 41233164
Submitted batch job 41233165
Submitted batch job 41233166
Submitted batch job 41233167
Submitted batch job 41233168
Submitted batch job 41233169
Submitted batch job 41233170
Submitted batch job 41233171
Submitted batch job 41233172
Submitted batch job 41233173
Submitted batch job 41233174
Submitted batch job 41233175
Submitted batc

In [11]:
! mkdir csec_intron_batch_gene

In [14]:
intron_mean_dfs = []
for i in os.listdir('csec_intron_batch_gene'):
    if i.endswith('Csec_intron_weighted_mean_rho.csv'):
        h = pd.read_csv(os.path.join('csec_intron_batch_gene', i))
        intron_mean_dfs.append(h)

In [16]:
pd.concat(intron_mean_dfs).to_csv('20230920_Csec_intron_mean_rho.csv')